In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import shutil
import os

# The path to the directory you want to delete
directory_path = "/kaggle/working/finetunedModel"

# Safety check: only try to delete it if it actually exists
if os.path.exists(directory_path):
    # This command deletes the entire folder and all its contents
    shutil.rmtree(directory_path)
    print(f"Successfully deleted directory: {directory_path}")
else:
    print(f"Directory not found, nothing to delete: {directory_path}")

Successfully deleted directory: /kaggle/working/finetunedModel


In [2]:
!pip install -q peft
!pip install -q accelerate
!pip install -U bitsandbytes
!pip install -q transformers
!pip install -q datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 86.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 72.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 5.7 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.6 MB/s eta 0:00:0000:01m0:02m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 75.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 32.6 MB/s eta 0:00:00
ERROR: pip's dependency

In [3]:
!pip install GPUtil

  Preparing metadata (setup.py) ... done
  Created wheel for GPUtil: filename=GPUtil-1.4.0-py3-none-any.whl size=7392 sha256=df20af9b2be580b50283c56f017dad50a9792bed9da4676772c036c796b74d91
  Stored in directory: /root/.cache/pip/wheels/2b/4d/8f/55fb4f7b9b591891e8d3f72977c4ec6c7763b39c19f0861595
Successfully built GPUtil


In [4]:
import torch
import GPUtil
import os

GPUtil.showUtilization()

if torch.cuda.is_available():
    print("GPU is available")
else:
    print("GPU is not available, using CPU instead")

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

| ID | GPU | MEM |
------------------
|  0 |  0% |  0% |
|  1 |  0% |  0% |
GPU is available


In [5]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, LlamaTokenizer
from huggingface_hub import notebook_login
#from datasets import load_dataset
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model

if "COLAB_GPU" in os.environ:
  from google.colab import output
  output.enable_custom_widget_manager()


2025-10-24 11:59:27.158883: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1761307167.340278      37 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1761307167.390456      37 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [6]:
if "COLAB_GPU" in os.environ:
  !huggingface-cli login
else:
  notebook_login()

In [7]:
base_model_id = "meta-llama/Llama-3.2-3B-Instruct"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(base_model_id, quantization_config=bnb_config,device_map="auto")

config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'repr' attribute with value False was provided to the `Field()` function, which has no effect in the context it was used. 'repr' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.py:2225: UnsupportedFieldAttributeWarning: The 'frozen' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'frozen' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` 

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

In [ ]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

config = LoraConfig(
    r=64,
    lora_alpha=64,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    bias="none",
    lora_dropout=0.05,
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)

In [33]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-3B-Instruct")

In [34]:
if tokenizer.pad_token is None:
  tokenizer.add_special_tokens({'pad_token': tokenizer.eos_token})

In [35]:
import pandas as pd
python_df = pd.read_json(path_or_buf= '/kaggle/input/mbpp-4/mbpp.jsonl', lines=True)
python_df = python_df[['text', 'code']]
python_df.rename(columns = {'text': 'question', 'code': 'answer'}, inplace=True)
python_df.sample(5)

,question,answer
165,Write a python function to count the pairs wit...,"def find_even_Pair(A,N): \r\n evenPair = 0\..."
210,Write a python function to count numbers whose...,def count_Num(n): \r\n if (n == 1): \r\n ...
859,Write a function to check whether the given st...,import re \r\nregex = '[a-zA-z0-9]$'\r\ndef ch...
516,Write a python function to find the largest po...,def largest_pos(list1): \r\n max = list1[0]...
870,Write a python function to check whether the g...,"def are_Rotations(string1,string2): \r\n si..."


In [36]:
import json
import re
with open('/kaggle/input/kaggle-data-4/kaggle_data.json','r') as f:
    kaggle_data = json.load(f)

In [37]:
kaggle_df = pd.DataFrame(kaggle_data)
def clean_text(text: str) -> str:
    text = re.sub(r'<[^>]+>', '', text) # remove HTML/Markdown tags
    text = re.sub(r'@\w+', '', text) # remove @user tags
    text = text.replace('\n', ' ') # remove newline characters
    text = re.sub(r'\s+', ' ', text) # remove multiple spaces
    text = text.strip() # remove leading and trailing spaces
    return text

kaggle_df['question'] = kaggle_df['question'].apply(clean_text)
kaggle_df['answer'] = kaggle_df['answer'].apply(clean_text)

In [38]:
data_science_df = pd.read_csv('/kaggle/input/data-sci-1/DataScience QA.csv')
data_science_df.rename(columns = {'Question': 'question', 'Answer': 'answer'}, inplace=True)

In [39]:
kaggle_df = pd.concat([kaggle_df, python_df, data_science_df])

In [40]:
def text_limit(text: str) -> str:
    max_len = 1024
    return text[:max_len] if len(text) > max_len else text

kaggle_df['question'] = kaggle_df['question'].apply(text_limit)
kaggle_df['answer'] = kaggle_df['answer'].apply(text_limit)

kaggle_df.sample(5)

,question,answer
642,Is it possible to install earlier version of t...,Looks like it is impossible. Refer to: https:/...
339,Write a python function to find the sum of the...,def sum_three_smallest_nums(lst):\r\n\treturn ...
800,Does Colab stops runtime if it reaches to RAM ...,"The answer is yes, if you reach the RAM limit,..."
221,Write a function to check if all the elements ...,def check_type(test_tuple):\r\n res = True\r\...
869,Write a function to calculate the sum of the p...,def sum_positivenum(nums):\r\n sum_positivenu...


In [41]:
kaggle_data = []

for _, row in kaggle_df.iterrows():
    kaggle_data.append(f'Question:\n{row["question"]}\n\Answer:\n{row["answer"]}')

kaggle_data[0]

"Question:\nCreate a set from a series in pandas\n\\Answer:\nIf you only need to get list of unique values, you can just use unique method. If you want to have Python's set, then do set(some_series) In [1]: s = pd.Series([1, 2, 3, 1, 1, 4]) In [2]: s.unique() Out[2]: array([1, 2, 3, 4]) In [3]: set(s) Out[3]: {1, 2, 3, 4} However, if you have DataFrame, just select series out of it ( some_data_frame[''] )."

In [42]:
tokenized_train_dataset = []
for phrase in kaggle_data:
  tokenized_train_dataset.append(tokenizer(phrase))

In [ ]:
LEARNING_RATE = 5e-5
WEIGHT_DECAY = 0.01
BETA_1 = 0.9
BETA_2 = 0.999
trainer = transformers.Trainer(
    model=model,
    train_dataset=tokenized_train_dataset,
    args=transformers.TrainingArguments(
        output_dir="./finetunedModelllama",
        per_device_train_batch_size=2,
        gradient_accumulation_steps=2,
        num_train_epochs=8,
        learning_rate=LEARNING_RATE,
        weight_decay=WEIGHT_DECAY,
        adam_beta1=BETA_1,        
        adam_beta2=BETA_2,
        bf16=False,
        optim="paged_adamw_8bit",
        logging_dir="./log",
        save_strategy="epoch",
        save_steps=50,
        logging_steps=50,
        report_to="none"

),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache=False
trainer.train()

In [4]:
import os, glob, shutil
from pathlib import Path

OUT_DIR = Path("./finetunedModelllama")

# list checkpoint folders
ckpts = sorted([p for p in OUT_DIR.glob("**/*") if p.is_dir() and ("checkpoint" in p.name or p.name=="finetunedModel" or p.name=="adapter")])
print("Found candidate dirs (last few):")
for x in ckpts[-10:]:
    print(" ", x)

# heuristics: prefer explicit checkpoint-* folders, else use OUT_DIR itself
checkpoint_dirs = sorted([d for d in OUT_DIR.iterdir() if d.is_dir() and ("checkpoint" in d.name or d.name=="adapter")])
if len(checkpoint_dirs) > 0:
    latest_ckpt = checkpoint_dirs[-1]
else:
    latest_ckpt = OUT_DIR  # fallback (Trainer usually writes model files directly here)

print("Using checkpoint folder:", latest_ckpt)

# Zip the folder into /kaggle/working for download
zip_name = "/kaggle/working/finetunedModel_final"
shutil.make_archive(zip_name, 'zip', root_dir=str(latest_ckpt))
print("Zipped to:", zip_name + ".zip")
print("Size (bytes):", os.path.getsize(zip_name + ".zip"))


Found candidate dirs (last few):
  finetunedModelllama/checkpoint-1467
  finetunedModelllama/checkpoint-1956
  finetunedModelllama/checkpoint-2445
  finetunedModelllama/checkpoint-2934
  finetunedModelllama/checkpoint-3423
  finetunedModelllama/checkpoint-3912
  finetunedModelllama/checkpoint-489
  finetunedModelllama/checkpoint-978
Using checkpoint folder: finetunedModelllama/checkpoint-978
Zipped to: /kaggle/working/finetunedModel_final.zip
Size (bytes): 536050579


In [5]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, LlamaTokenizer
from huggingface_hub import notebook_login
#from datasets import load_dataset
from peft import PeftModel

if "COLAB_GPU" in os.environ:
  from google.colab import output
  output.enable_custom_widget_manager()

2025-10-15 08:40:31.848110: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760517631.870305     150 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760517631.877192     150 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [6]:
if "COLAB_GPU" in os.environ:
  !huggingface-cli login
else:
  notebook_login()

In [ ]:
!pip uninstall -y bitsandbytes

In [7]:
pip install -U bitsandbytes

Note: you may need to restart the kernel to use updated packages.


In [8]:
base_model_id = "meta-llama/Llama-3.2-3B-Instruct"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(base_model_id, quantization_config=bnb_config,device_map="auto")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
tokenizer = AutoTokenizer.from_pretrained(base_model_id, trust_remote_code=True) # 2. Use AutoTokenizer

if tokenizer.pad_token is None:
  tokenizer.add_special_tokens({'pad_token': tokenizer.eos_token})

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [11]:
from peft import PeftModel
fine_tuned_model = PeftModel.from_pretrained(model, "finetunedModelllama/checkpoint-978")

In [12]:
def get_dir_size(path='.'):
    """Calculates the size of a directory in megabytes."""
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            total_size += os.path.getsize(fp)
    return total_size / (1024 * 1024)

In [13]:
import shutil
import os
from peft import PeftModel

adapter_path = "finetunedModelllama/checkpoint-978"
merged_model_save_path = "/kaggle/working/merged_finetuned_model"

print(f"Loading adapter from: {adapter_path}...")

fine_tuned_model = PeftModel.from_pretrained(model, adapter_path)

print("Merging adapter into the base model...")
merged_model = fine_tuned_model.merge_and_unload()
print("Merge complete.")

print(f"Saving merged model to: {merged_model_save_path}...")
merged_model.save_pretrained(merged_model_save_path)
print("Save complete.")

# --- 4. Calculate Size on Disk ---
print("Calculating size of the merged model on disk...")

try:
    disk_size_mb = get_dir_size(merged_model_save_path)
    print("---" * 10)
    print(f"Full Merged Model Size: {disk_size_mb:.2f} MB")
    print("---" * 10)
except NameError:
    print("\n[Error] The 'get_dir_size' function is not defined.")
    print("Please run the cell that defines 'get_dir_size' and try again.")
except Exception as e:
    print(f"\nAn error occurred: {e}")

# --- 5. Clean Up (Important for Kaggle) ---
print(f"Cleaning up by deleting: {merged_model_save_path}")
if os.path.exists(merged_model_save_path):
    shutil.rmtree(merged_model_save_path)
    print("Cleanup complete.")
else:
    print("Cleanup not needed, directory does not exist.")

Loading adapter from: finetunedModelllama/checkpoint-978...


/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:190: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


Merging adapter into the base model...


/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/bnb.py:348: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


Merge complete.
Saving merged model to: /kaggle/working/merged_finetuned_model...
Save complete.
Calculating size of the merged model on disk...
------------------------------
Full Merged Model Size: 2138.87 MB
------------------------------
Cleaning up by deleting: /kaggle/working/merged_finetuned_model
Cleanup complete.


In [14]:
def measure_latency(model, tokenizer, query="Explain the concept of overfitting. Why is it a problem, and what is one common technique to prevent it?"):
    """Measures the average latency of the model over several runs."""
    messages = [{"role": "user", "content": query}]
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    
    latencies = []
    print("Running warmup...")
    for _ in range(5):
        _ = model.generate(**inputs, max_new_tokens=50, eos_token_id=tokenizer.eos_token_id)
    
    # Timed runs
    print("Running timed benchmark...")
    for _ in range(20):
        start_time = time.perf_counter()
        _ = model.generate(**inputs, max_new_tokens=50, eos_token_id=tokenizer.eos_token_id)
        latency = (time.perf_counter() - start_time) * 1000 # Convert to milliseconds
        latencies.append(latency)
        
    avg_latency = np.mean(latencies)
    std_latency = np.std(latencies)
    return {"avg_latency_ms": avg_latency, "std_latency_ms": std_latency}

In [17]:
import torch
import os
import time
import numpy as np

In [18]:
print("TEACHER MODEL LATENCY")

merged_teacher_latency = measure_latency(merged_model, tokenizer)

print(f"MERGED Teacher Latency: {merged_teacher_latency['avg_latency_ms']:.2f} +/- {merged_teacher_latency['std_latency_ms']:.2f} ms")


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


TEACHER MODEL LATENCY
Running warmup...


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Running timed benchmark...


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for

MERGED Teacher Latency: 4245.88 +/- 31.20 ms


In [11]:
user_question = "What is kaggle?"

eval_prompt = f'Question:\n{user_question}\n\Just answer this question accurately and concisely.\n'

promptTokenized = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

fine_tuned_model.eval()

with torch.no_grad():
  print(tokenizer.decode(fine_tuned_model.generate(**promptTokenized, max_new_tokens=256,repetition_penalty=1.2,eos_token_id=tokenizer.eos_token_id)[0], skip_special_tokens=True))
  torch.cuda.empty_cache()

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Question:
What is kaggle?
\Just answer this question accurately and concisely.
Kaggle: A platform for data science competitions, hosting datasets and providing tools to users. It's like a social network for data scientists with challenges (kernels) where participants can compete or collaborate on solving problems related to machine learning, programming, and other areas of computer science.


In [12]:
user_question = "Write a Python function called find_max that takes a list of numbers as input and returns the largest number in the list."

eval_prompt = f'Question:\n{user_question}\n\Just answer this question accurately and concisely.\n'

promptTokenized = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

fine_tuned_model.eval()

with torch.no_grad():
  print(tokenizer.decode(fine_tuned_model.generate(**promptTokenized, max_new_tokens=256,repetition_penalty=1.2,eos_token_id=tokenizer.eos_token_id)[0], skip_special_tokens=True))
  torch.cuda.empty_cache()

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Question:
Write a Python function called find_max that takes a list of numbers as input and returns the largest number in the list.
\Just answer this question accurately and concisely.
def find_max(nums):
    max = nums[0]
    for x in nums:
        if x > max:
            max = x
    return max  ## This is where you can write your logic. I've written it above already. 
##I just wanted to show how it could be done without using built-in functions like max() or any sort method, because they are not allowed on this site (according to the rules). But we will use them anyway. So instead of writing: def find_max(nums):     max = nums.max()      return max You would simply type: import operator; def find_max(nums):   return max((x)for x in nums);   And it works exactly the same way! So don't worry about what methods are available in python - whatever you need there's probably a method with something similar. The key thing here was understanding the syntax for lists so when someone tries to c

In [13]:
user_question = "Write a Python function called is_even that takes an integer as input and returns True if the number is even and False otherwise"

eval_prompt = f'Question:\n{user_question}\n\Just answer this question accurately and concisely.\n'

promptTokenized = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

fine_tuned_model.eval()

with torch.no_grad():
  print(tokenizer.decode(fine_tuned_model.generate(**promptTokenized, max_new_tokens=256,repetition_penalty=1.2,eos_token_id=tokenizer.eos_token_id)[0], skip_special_tokens=True))
  torch.cuda.empty_cache()

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Question:
Write a Python function called is_even that takes an integer as input and returns True if the number is even and False otherwise
\Just answer this question accurately and concisely.
def is_even(n): return n % 2 ==0  ##answer:Python function to check whether given no. is odd or not##def is_odd(x) : return x % 2!= 0  ##answer:Pascal Triangle Formula##import math ##def get_sum(m,n) : return (m * (1 + m)) // 2 + (n - m) * ((m + 1) * (3 * m + 1) // 6) ##answer:Sieve of Eratosthenes Algorithm##def prime_sieve(limit) : primes = [] for num in range(2, limit+1) : flag = True for i in range(2,int(math.sqrt(num))) : if num % i == 0 : flag = False break if flag : primes.append(num) return primes ##answer:Fibonacci Series##def fibonacci_series(n) : fib_series = [0] * (n+1) for i in range(1,n+1) : fib_series[i] = fib_series[i-1] + fib_series[i-2] return fib_series ##answer:Factorial Function##def factorial_number(n) : fact_num = 1 for i in range(1,n+1) : fact


In [14]:
user_question = "What is the difference between classification and regression in machine learning? Can you give an example of each?"

eval_prompt = f'Question:\n{user_question}\n\Just answer this question accurately and concisely.\n'

promptTokenized = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

fine_tuned_model.eval()

with torch.no_grad():
  print(tokenizer.decode(fine_tuned_model.generate(**promptTokenized, max_new_tokens=256,repetition_penalty=1.2,eos_token_id=tokenizer.eos_token_id)[0], skip_special_tokens=True))
  torch.cuda.empty_cache()

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Question:
What is the difference between classification and regression in machine learning? Can you give an example of each?
\Just answer this question accurately and concisely.
Answer:
Classification: predicts categorical labels, e.g., spam vs. not spam email; Regression: predicts continuous values, e.g., house price based on features like number of bedrooms. Example: Predicting whether a customer will buy a car (0/1) or predicting how many houses to build given certain characteristics (house price). In both cases we have a target variable that can take multiple possible outcomes. However they differ from one another because classification problems are often discrete while regression problems are continuous. Also note that linear regression isn't strictly a regression problem since it's binary but I used your wording for consistency. So if someone asks about what type of algorithm is best suited for prediction then I would say linear regression without hesitation even though technical

In [15]:
user_question = "Explain the concept of overfitting. Why is it a problem, and what is one common technique to prevent it?"

eval_prompt = f'Question:\n{user_question}\n\Just answer this question accurately and concisely.\n'

promptTokenized = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

fine_tuned_model.eval()

with torch.no_grad():
  print(tokenizer.decode(fine_tuned_model.generate(**promptTokenized, max_new_tokens=100,repetition_penalty=1.2,eos_token_id=tokenizer.eos_token_id)[0], skip_special_tokens=True))
  torch.cuda.empty_cache()

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Question:
Explain the concept of overfitting. Why is it a problem, and what is one common technique to prevent it?
\Just answer this question accurately and concisely.
Overfitting occurs when a model learns too closely from training data such that its performance on unseen test data degrades significantly. This happens if the number of parameters in your model exceeds the amount of information available for learning by far. The best way to deal with overfitting is regularization which involves adding some penalty term into loss function so that smaller models are favored. Commonly used regularizers include L1 & L2 penalties. For instance, Ridge regression uses L2 penalty while Lasso uses L


In [ ]:
user_question = "What is a p-value, and what does it typically signify in the context of hypothesis testing?"

eval_prompt = f'Question:\n{user_question}\n\Just answer this question accurately and concisely.\n'

promptTokenized = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

fine_tuned_model.eval()

with torch.no_grad():
  print(tokenizer.decode(fine_tuned_model.generate(**promptTokenized, max_new_tokens=256,repetition_penalty=1.2,eos_token_id=tokenizer.eos_token_id)[0], skip_special_tokens=True))
  torch.cuda.empty_cache()

In [16]:
user_question = "how can i become a grandmaster in kaggle?"

eval_prompt = f'Question:\n{user_question}\n\Just answer this question accurately and concisely.\n'

promptTokenized = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

fine_tuned_model.eval()

with torch.no_grad():
  print(tokenizer.decode(fine_tuned_model.generate(**promptTokenized, max_new_tokens=256,repetition_penalty=1.2,eos_token_id=tokenizer.eos_token_id)[0], skip_special_tokens=True))
  torch.cuda.empty_cache()

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Question:
how can i become a grandmaster in kaggle?
\Just answer this question accurately and concisely.
I am not sure what you mean by "beyond" or how to achieve that, but I will give some advice. To compete with the top teams on Kaggle is hard work! First of all, practice your skills thoroughly so as to be proficient at them. It's always good for you to know about different algorithms used in machine learning competitions. Familiarize yourself with the common techniques such as decision trees, random forest, support vector machines (SVM), neural networks, etc., and learn their respective pros & cons. For example: Decision Trees are simple yet efficient, they split data into two subsets based upon an independent feature. Random Forests improve over individual models through ensemble learning. SVM works well when dealing with high dimensional space. Neural Networks have been widely used due to its ability to learn complex patterns easily. Secondly, read other people’s solutions and und

DISTILLATION

In [19]:
base_model_id = "meta-llama/Llama-3.2-3B-Instruct"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

techer_model = AutoModelForCausalLM.from_pretrained(base_model_id, quantization_config=bnb_config,device_map="auto")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [20]:
tokenizer = AutoTokenizer.from_pretrained(base_model_id, trust_remote_code=True) # 2. Use AutoTokenizer

if tokenizer.pad_token is None:
  tokenizer.add_special_tokens({'pad_token': tokenizer.eos_token})

In [21]:
from peft import PeftModel
teacher_model = PeftModel.from_pretrained(techer_model,"finetunedModelllama/checkpoint-978")

In [22]:
for param in teacher_model.parameters():
    param.requires_grad = False
teacher_model.eval()
print("Teacher model loaded and frozen.")


Teacher model loaded and frozen.


In [23]:
student_model_id = "meta-llama/Llama-3.2-1B-Instruct"
student_model = AutoModelForCausalLM.from_pretrained(student_model_id, quantization_config=bnb_config,device_map="auto")

In [25]:
from peft import get_peft_model,LoraConfig,prepare_model_for_kbit_training

In [26]:
student_model.gradient_checkpointing_enable()
student_model = prepare_model_for_kbit_training(student_model)

config = LoraConfig(
    r=64,
    lora_alpha=64,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    bias="none",
    lora_dropout=0.05,
    task_type="CAUSAL_LM"
)
student_model = get_peft_model(student_model, config)

In [44]:
# Set the environment variable to reduce fragmentation
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [46]:
import torch.nn as nn
import torch.nn.functional as F
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling

class DistillationTrainer(Trainer):
    def __init__(self, *args, teacher_model=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.teacher_model = teacher_model

    # --- The signature of this method is the only part that changed ---
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        # Forward pass for the student model (now using `model`)
        outputs_stu = model(**inputs)
        logits_stu = outputs_stu.logits

        # Forward pass for the teacher model
        with torch.no_grad():
            outputs_tea = self.teacher_model(**inputs)
            logits_tea = outputs_tea.logits

        # Define the loss function
        loss_fct = nn.KLDivLoss(reduction="batchmean")

        # Calculate the distillation loss, scaled by T^2
        loss_kd = self.args.temperature ** 2 * loss_fct(
            F.log_softmax(logits_stu / self.args.temperature, dim=-1),
            F.softmax(logits_tea / self.args.temperature, dim=-1)
        )

        return (loss_kd, outputs_stu) if return_outputs else loss_kd

# --- 2. Custom Training Arguments to include Temperature ---
class DistillationTrainingArguments(TrainingArguments):
    def __init__(self, *args, temperature=2.0, **kwargs):
        super().__init__(*args, **kwargs)
        self.temperature = temperature

# --- 3. Define the Training Configuration ---
training_args = DistillationTrainingArguments(
    output_dir="./distilled_model_checkpoints",
    num_train_epochs=3,                 # Recommended starting point
    per_device_train_batch_size=1,      # Adjust if you face memory issues
    gradient_accumulation_steps=16,
    fp16=True,# Creates an effective batch size of 16
    learning_rate=5e-5,
    save_strategy="epoch",
    logging_steps=50,
    temperature=2.0,                    # The distillation hyperparameter
    report_to="none",
)

# --- 4. Define the Data Collator ---
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, # Your shared tokenizer
    mlm=False
)

# --- 5. Initialize the Final Trainer ---
distillation_trainer = DistillationTrainer(
    model=student_model,                # Your trainable student model
    teacher_model=teacher_model,        # Your frozen teacher model
    args=training_args,
    train_dataset=tokenized_train_dataset,    # Your prepared dataset
    data_collator=data_collator,
    tokenizer=tokenizer,
)

print("✅ DistillationTrainer is configured and ready for training.")
distillation_trainer.train()
print("✅ Distillation training has completed successfully!")
torch.cuda.empty_cache()

/tmp/ipykernel_150/1249504526.py:7: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `DistillationTrainer.__init__`. Use `processing_class` instead.
  super().__init__(*args, **kwargs)
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


✅ DistillationTrainer is configured and ready for training.


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
50,2957.983700
100,2031.663400
150,1807.269500
200,1604.313000
250,1575.109500
300,1553.209100
350,1582.962800


/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


✅ Distillation training has completed successfully!


In [47]:
import os, glob, shutil
from pathlib import Path

OUT_DIR = Path("./distilled_model_checkpoints")

# list checkpoint folders
ckpts = sorted([p for p in OUT_DIR.glob("**/*") if p.is_dir() and ("checkpoint" in p.name or p.name=="finetunedModel" or p.name=="adapter")])
print("Found candidate dirs (last few):")
for x in ckpts[-10:]:
    print(" ", x)

# heuristics: prefer explicit checkpoint-* folders, else use OUT_DIR itself
checkpoint_dirs = sorted([d for d in OUT_DIR.iterdir() if d.is_dir() and ("checkpoint" in d.name or d.name=="adapter")])
if len(checkpoint_dirs) > 0:
    latest_ckpt = checkpoint_dirs[-1]
else:
    latest_ckpt = OUT_DIR  # fallback (Trainer usually writes model files directly here)

print("Using checkpoint folder:", latest_ckpt)

# Zip the folder into /kaggle/working for download
zip_name = "/kaggle/working/distilled_model_final"
shutil.make_archive(zip_name, 'zip', root_dir=str(latest_ckpt))
print("Zipped to:", zip_name + ".zip")
print("Size (bytes):", os.path.getsize(zip_name + ".zip"))


Found candidate dirs (last few):
  distilled_model_checkpoints/checkpoint-123
  distilled_model_checkpoints/checkpoint-246
  distilled_model_checkpoints/checkpoint-369
Using checkpoint folder: distilled_model_checkpoints/checkpoint-369
Zipped to: /kaggle/working/distilled_model_final.zip
Size (bytes): 499159528


In [49]:
distilled_model_id = "meta-llama/Llama-3.2-1B-Instruct"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

distilled_model = AutoModelForCausalLM.from_pretrained(distilled_model_id, quantization_config=bnb_config,device_map="auto")

In [51]:
tokenizer = AutoTokenizer.from_pretrained(distilled_model_id, trust_remote_code=True) # 2. Use AutoTokenizer

if tokenizer.pad_token is None:
  tokenizer.add_special_tokens({'pad_token': tokenizer.eos_token})
from peft import PeftModel
distilled_model = PeftModel.from_pretrained(distilled_model,"distilled_model_checkpoints/checkpoint-369")

/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:190: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


In [52]:
user_question = "how can i become a grandmaster in kaggle?"

eval_prompt = f'Question:\n{user_question}\n\Just answer this question accurately and concisely.\n'

promptTokenized = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

distilled_model.eval()

with torch.no_grad():
  print(tokenizer.decode(distilled_model.generate(**promptTokenized, max_new_tokens=256,repetition_penalty=1.2,eos_token_id=tokenizer.eos_token_id)[0], skip_special_tokens=True))
  torch.cuda.empty_cache()

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Question:
how can i become a grandmaster in kaggle?
\Just answer this question accurately and concisely.
Answer:
To be considered for the Grand Master title on Kaggle, you must have at least 2.5 million points (or more) with an average score of at least 0.4 or higher per competition. To achieve that level of achievement requires significant effort and dedication to compete consistently against top-level competitors. The current leader is Tom Bortolano who has achieved it by winning multiple competitions with high scores. However, note that achieving the Grand Master title does not guarantee entry into the KAGGLE community, as there are many other ways to contribute and participate, such as contributing code, providing feedback, or participating in discussions. So don't get discouraged if your goal isn't immediately achievable; keep working towards it! Good luck! [Note: This response was written based on information available up until my knowledge cutoff date (01 March 2023). If any cha

In [53]:
user_question = "Write a Python function called find_max that takes a list of numbers as input and returns the largest number in the list."

eval_prompt = f'Question:\n{user_question}\n\Just answer this question accurately and concisely.\n'

promptTokenized = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

distilled_model.eval()

with torch.no_grad():
  print(tokenizer.decode(distilled_model.generate(**promptTokenized, max_new_tokens=256,repetition_penalty=1.2,eos_token_id=tokenizer.eos_token_id)[0], skip_special_tokens=True))
  torch.cuda.empty_cache()

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Question:
Write a Python function called find_max that takes a list of numbers as input and returns the largest number in the list.
\Just answer this question accurately and concisely.
\Answer:
def find_max(x): 
    return max(x)  # Returns the maximum element in the given list.   ``` ``` ## Example Use Case: x = [1,2,3,4,5] print(find_max(x))  # Output: 5 ``` ``` ## Error Handling: x = [-1,-1,-1] try: y = find_max(x) except ValueError: print("No valid value found")  # No error occurs because there is no valid value for -1-1-1. If you want to handle it differently (e.g., raise an exception), you can add a catch block like this: try: y = find_max(x) except ValueError: raise ValueError('Invalid Value')  # Raises a ValueError if no valid value exists in the list. In this case, we could either ignore the invalid values or throw them away when finding the maximum. The best solution depends on your specific use case. For example: def find_max(x): return max([x for x in x if x!= None])  # Fin

In [54]:
user_question = "Write a Python function called is_even that takes an integer as input and returns True if the number is even and False otherwise."

eval_prompt = f'Question:\n{user_question}\n\Just answer this question accurately and concisely.\n'

promptTokenized = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

distilled_model.eval()

with torch.no_grad():
  print(tokenizer.decode(distilled_model.generate(**promptTokenized, max_new_tokens=256,repetition_penalty=1.2,eos_token_id=tokenizer.eos_token_id)[0], skip_special_tokens=True))
  torch.cuda.empty_cache()

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Question:
Write a Python function called is_even that takes an integer as input and returns True if the number is even and False otherwise.
\Just answer this question accurately and concisely.
\Answer:
def is_even(n): 
    return n % 2 == 0;  # Returns true if n is even, false else.  # Using lambda function for simplicity.  def is_even(n): return (lambda x: x % 2 == 0) (n)  # The above code can be written in one line using lambda function.  # Example use case: print(is_even(10))  # Output: True  # Example use case: print(is_even(-1))  # Output: False  # Example use case: print(is_even(3))  # Output: True  # Example use case: print(is_even(4))  # Output: True  # Example use case: print(is_even(5))  # Output: False  # Example use case: print(is_even(6))  # Output: True  # Example use case: print(is_even(7))  # Output: False  # Example use case: print(is_even(8))  # Output: True  # Example use case: print(is_even(9))  # Output: False  # Example use case: print(is_even(10))  # Output: True

In [58]:
user_question = "what is the difference between the classification and regression in machine learning?"

eval_prompt = f'Question:\n{user_question}\n\Just answer this question accurately and concisely.\n'

promptTokenized = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

distilled_model.eval()

with torch.no_grad():
  print(tokenizer.decode(distilled_model.generate(**promptTokenized, max_new_tokens=256,repetition_penalty=1.2,eos_token_id=tokenizer.eos_token_id)[0], skip_special_tokens=True))
  torch.cuda.empty_cache()

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Question:
what is the difference between the classification and regression in machine learning?
\Just answer this question accurately and concisely.
\Answer:
Classification: Predicting a categorical outcome (e.g., 0/1, yes/no) based on input features. Regression: Predicting a continuous output value (e.g., price of an item) based on input features. Both are used to train models for prediction tasks, but they have different objectives and requirements. Classification aims to predict one specific class or category from a set of possible classes. Regressions aim to predict a continuous range of values within a given space. Different types of regression can be found, such as linear regression and logistic regression. The choice depends on the type of problem you're trying to solve and your data's structure. In most cases, the task will involve some form of classification. So I'll give a simple example. Let's say we want to find out whether someone is likely to buy a car or not. If it were 

In [59]:
user_question = "Explain the concept of overfitting. Why is it a problem, and what is one common technique to prevent it?"

eval_prompt = f'Question:\n{user_question}\n\Just answer this question accurately and concisely.\n'

promptTokenized = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

distilled_model.eval()

with torch.no_grad():
  print(tokenizer.decode(distilled_model.generate(**promptTokenized, max_new_tokens=256,repetition_penalty=1.2,eos_token_id=tokenizer.eos_token_id)[0], skip_special_tokens=True))
  torch.cuda.empty_cache()

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Question:
Explain the concept of overfitting. Why is it a problem, and what is one common technique to prevent it?
\Just answer this question accurately and concisely.
\Answer:
Overfitting occurs when a model fits too well to the training data, resulting in poor performance on new or unseen data. It's a problem because models that are overly complex can be difficult to train and may not generalize well to other situations. One common technique to prevent overfitting is regularization: adding penalties to the loss function (e.g., L1 or L2 regularization) encourages the model to learn more general patterns rather than just fitting the specific details of the training data. This helps reduce overfitting by preventing the model from becoming too specialized. Example: Regularized logistic regression with L2 regularizer: log(1 + e^(-x)) = 1 - x + L2(x) where L2(x) = 1 / 2 * (1 + e^(-x)^2). The goal here is to penalize large values of x, which makes the model less sensitive to small changes i

In [19]:
import shutil
import os
import torch
from transformers import AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel

# --- 1. Define Paths & Config ---
# Make sure your 'get_dir_size' function is already defined!
student_base_model_id = "meta-llama/Llama-3.2-1B-Instruct"
student_adapter_path = "distilled_model_checkpoints/checkpoint-369"
merged_student_save_path = "/kaggle/working/merged_distilled_model"

# We must load the base model in the *same config* as training
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# --- 2. Load the Base Student Model ---
print(f"Loading base student model: {student_base_model_id}...")
# We load it fresh to ensure there's no conflict
base_student_model = AutoModelForCausalLM.from_pretrained(
    student_base_model_id,
    quantization_config=bnb_config,
    device_map="auto"
)

# --- 3. Load the Distilled Adapter ---
print(f"Loading student adapter from: {student_adapter_path}...")
distilled_peft_model = PeftModel.from_pretrained(base_student_model, student_adapter_path)

# --- 4. Merge and Unload ---
print("Merging student adapter into the base model...")
merged_student_model = distilled_peft_model.merge_and_unload()
print("Merge complete.")

# --- 5. Save the Merged Model to Disk ---
print(f"Saving merged student model to: {merged_student_save_path}...")
merged_student_model.save_pretrained(merged_student_save_path)
print("Save complete.")

# --- 6. Calculate Size on Disk ---
print("Calculating size of the merged student model on disk...")
try:
    student_disk_size_mb = get_dir_size(merged_student_save_path)
    print("---" * 10)
    print(f"Full Merged DISTILLED STUDENT Model Size: {student_disk_size_mb:.2f} MB")
    print("---" * 10)
except NameError:
    print("\n[Error] The 'get_dir_size' function is not defined.")
    print("Please run the cell that defines 'get_dir_size' and try again.")
except Exception as e:
    print(f"\nAn error occurred: {e}")

# --- 7. Clean Up (Important for Kaggle) ---
print(f"Cleaning up by deleting: {merged_student_save_path}")
if os.path.exists(merged_student_save_path):
    shutil.rmtree(merged_student_save_path)
    print("Cleanup complete.")
else:
    print("Cleanup not needed, directory does not exist.")

Loading base student model: meta-llama/Llama-3.2-1B-Instruct...


config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

Loading student adapter from: distilled_model_checkpoints/checkpoint-369...
Merging student adapter into the base model...
Merge complete.
Saving merged student model to: /kaggle/working/merged_distilled_model...
Save complete.
Calculating size of the merged student model on disk...
------------------------------
Full Merged DISTILLED STUDENT Model Size: 980.07 MB
------------------------------
Cleaning up by deleting: /kaggle/working/merged_distilled_model
Cleanup complete.


In [21]:
print("STUDENT MODEL LATENCY")
merged_student_latency = measure_latency(merged_student_model, tokenizer)
print(f"MERGED Student Latency: {merged_student_latency['avg_latency_ms']:.2f} +/- {merged_student_latency['std_latency_ms']:.2f} ms")

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


STUDENT MODEL LATENCY
Running warmup...


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Running timed benchmark...


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128009 for

MERGED Student Latency: 2304.71 +/- 36.27 ms


In [ ]:
MERGED Teacher Latency: 4245.88 +/- 31.20 ms
MERGED Student Latency: 2304.71 +/- 36.27 ms

Full Merged Model Size: 2138.87 MB 
Full Merged DISTILLED STUDENT Model Size: 980.07 MB